In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }

params = code_gas.Params(**params_dict, H_r_model='irradiated')
print("model", params.H_r_model)
m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params), params)
print('m0', m0)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)


peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

sim = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params, sim_params=sim_params)



In [ ]:
fig,axs = plt.subplots(2,2, figsize = (18,18))


######### dR/dt #########
axs[1,0].scatter(sim.time.to(u.Myr), sim.dR_dt[0], color = 'black')
axs[1,0].set_xscale('log')
axs[1,0].set_xlabel('t [Myr]', size = 20)
axs[1,0].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)

######### dR/dt #########
axs[0,1].scatter(sim.position.to(u.au)[0], sim.dR_dt[0], color = 'black')
axs[0,1].set_xscale('log')
axs[0,1].set_xlabel('R [AU]', size = 20)
axs[0,1].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)
print("dr/dt", sim.dR_dt[0])
print("r/rdot", sim.position.to(u.au)[0]/sim.dR_dt[0])
print("step size [yr]", (sim_params.step_size*u.Myr))
print("time vector[Myr]", (sim.time*u.Myr))
print(sim.time.size)
print(np.where(sim_params.step_size*u.Myr>(sim.position[0]/np.abs(sim.dR_dt[0])))[0])
print("min incremento", np.min(sim.position[0]/np.abs(sim.dR_dt[0])))
print("min incremento massa", np.min(sim.mass[0]/np.abs(sim.dM_dt[0])))

print("initial mass", sim.mass[0][0])
############ dM/dt #########
axs[0,0].scatter(sim.time.to(u.Myr), sim.dM_dt[0], color = 'black')
axs[0,0].set_xscale('log')
axs[0,0].set_xlabel('t [Myr]]', size = 20)
axs[0,0].set_ylabel('$\dot{M}$ [$M_{\oplus}$/Myr]', size = 20)
axs[0,0].axhline(sim_params.step_size, color = 'red', linestyle = '--')
axs[0,0].set_xlim(0.2, 0.3)
axs[1,0].set_xlim(0.2, 0.3)


############ growth track ##############
#plot the magnetic cavity and shade the region inside it (from initial to final position)
axs[1,1].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.05)
axs[1,1].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.05)
axs[1,1].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =0.05)
a_p0 = np.geomspace(1e-3, 1e2, num = 1000)
m0 = M0_pla(a_p0, sim_params.t_in, sigma_gas_steady_state(a_p0, sim_params.t_in, params), params)
axs[1,1].loglog(a_p0, m0, linestyle  = ':', color = 'lightblue', zorder = 0)
norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
cmap = mpl.colormaps["inferno"].reversed()
for p in range(sim_params.nr_planets-1,-1,-1):
    stop_mig_indices = np.where(sim.dR_dt[p] == 0)[0]
    if stop_mig_indices.size > 0:
        stop_mig_idx = stop_mig_indices[0]
    else:
        stop_mig_idx = len(sim.dR_dt[p]) - 1  # or some other default value

    isolation_mass = M_peb_iso(sim.position[p].value, sim.time.value, params)
    isolation_idx = np.argmax(sim.mass[p].value > isolation_mass)
    axs[1,0].axvline(sim.time[isolation_idx].value, color = 'lightblue', linestyle = '--')

    if isolation_idx < sim.mass[p].size:
        dt= (sim.time[:isolation_idx]).to(u.Myr)
        sc = axs[1,1].scatter(sim.position.to(u.au)[p,:isolation_idx], sim.mass[p,:isolation_idx].to(u.earthMass), c=dt, norm=norm,  cmap = cmap)
        axs[1,1].scatter(sim.position.to(u.au)[p,isolation_idx], sim.mass[p,isolation_idx].to(u.earthMass), color = 'black', s = 150, zorder = 100)
    else: 
        dt= (sim.time[:stop_mig_idx]).to(u.Myr)
        sc = axs[1,1].scatter(sim.position.to(u.au)[p,:stop_mig_idx], sim.mass[p,:stop_mig_idx].to(u.earthMass), c=dt, norm=norm,  cmap = cmap)

        #axs.scatter(sim.position.to(u.au)[-1],sim.mass[-1].to(u.earthMass), marker = 'x', color = 'grey',s = 100, zorder=100)
    axs[1,1].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
    sc_post_iso = axs[1,1].loglog(sim.position.to(u.au)[p,isolation_idx:stop_mig_idx], sim.mass[p,isolation_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, alpha = 0.1)
axs[1,1].set_ylim(1e-7, 7e2)
axs[1,1].set_xlim(5e-3, 1e2)
axs[1,1].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
axs[1,1].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
plot.all_y_ticks(axs[1,1], num_ticks=100)

cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(sc, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0, ]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)


for i in range(2):
    for j in range(2):
        axs[i,j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
        axs[i,j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)

plt.savefig("figures/t_fixed_adaptive_2au", bbox_inches='tight')

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'M_dot_star': None,
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1e-2,
                'epsilon_heat':0.5,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'H_r_model': "irradiated", 
                }
params = code_gas.Params(**params_dict)
t_fin = 3 #Myr, end of sim

N_steps = 10000
import scipy.stats as stats

# Number of samples to generate
num_samples = 100
seed = 42
seed_t0 = 11
see_ap0 = 7

#inner embryo
t0_inner_samples = stats.uniform.rvs(0.1, 1, size=num_samples, random_state=seed_t0)
print(t0_inner_samples)
a_p0_inner_samples = stats.loguniform.rvs(1e-1, 1e1, size=num_samples, random_state=see_ap0)
t0_inner = ([t0_inner_samples] * np.ones(len(a_p0_inner_samples))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
# outer embryo
a_p0_outer = np.array([30])
t0_outer = ([1] * np.ones(len(a_p0_outer)) ) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
sigma_gas_outer = sigma_gas_steady_state(a_p0_outer, t0_outer, params)
m0_outer = M0_pla(a_p0_outer, t0_outer, sigma_gas_outer, params)

fig,axs = plt.subplots(1,1, figsize=(6,6))
axs.scatter(a_p0_inner_samples, t0_inner_samples, label='Inner planet', color='blue')
#axs.set_xscale('log')

simulations = []
parameters = []
sim_parameters = []

for  a_p0_in, t0_in in tqdm(zip(a_p0_inner_samples, t0_inner_samples)):
    #initial conditions: both a_p0 and m0 take the outer planet and one of the inner planets
    sigma_gas_inner = sigma_gas_steady_state(a_p0_in, t0_in, params)
    m0_in = M0_pla(a_p0_in, t0_in, sigma_gas_inner, params)
 
    a_p0 = np.array([*a_p0_outer, a_p0_in])
    m_0 = np.array([*m0_outer, m0_in])
    t0 = np.array([*t0_outer, t0_in])
    print(a_p0, m_0, t0)

    sim_params_dict = {'N_step': N_steps,
                    'm0': m_0,
                    'a_p0': a_p0,
                    't0': t0,
                    't_fin': t_fin,
                }
    sim_params = code_gas.SimulationParams(**sim_params_dict)
    sim_parameters.append(sim_params)


fig, axs = plt.subplots(1,3, figsize=(18, 6))
title = ["Irradiation", "Surface heating", "Midplane heating"]
a_p0 = np.geomspace(5e-3, 1e2, num = 1000)

alpha_transp = 0.2


import json
import os
folder_path = "sims/gas_acc"

#lists all files in the folder
all_files = os.listdir(folder_path)
# filters in case there is sth which is not JSON
filenames = [os.path.join(folder_path, f) for f in all_files if f.endswith('.json')]

def JSONtoSimRes(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    
    simulation_results = code_gas.SimulationResults(
        time=np.array(data['time']) * u.Myr,
        mass=np.array(data['mass']) * u.M_earth,
        position=np.array(data['position']) * u.au,
        dM_dt=np.array(data['dM_dt']) * (u.M_earth / u.Myr),
        dR_dt=np.array(data['dR_dt']) * (u.au / u.Myr),
        filter_fraction=np.array(data['filter_fraction']),
        flux_on_planet=np.array(data['flux_on_planet']),
        F0=np.array(data['F0']),
        flux_ratio=np.array(data['flux_ratio']),
        sigma_peb=np.array(data['sigma_peb']),
        sigma_gas=np.array(data['sigma_gas']),
        acc_regimes=np.array(data['acc_regimes']),
        gas_acc_dict=data['gas_acc_dict']
    )
    
    return simulation_results

simulations = [JSONtoSimRes(filename) for filename in filenames]

############### DISCS loops ########################
for k, (simulations, parameters, sim_parameters) in enumerate(zip([simulations], [params], [sim_parameters])):
#for k, (simulations, parameters, sim_parameters) in enumerate(zip([simulations, simulations_ice, simulations_alpha], [parameters, parameters_ice, parameters_alpha], [sim_parameters, sim_parameters_ice, sim_parameters_alpha])):
    for i in range(len(simulations)):

        sim = simulations[i]
        #params = parameters[i]
        sim_params = sim_parameters[i]
        for p in range(1,sim_params.nr_planets):
            print("time vector", sim.time[0], "mass vector planet", sim.mass[p])


            idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            cmap =  mpl.cm.inferno.reversed()
            norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
            colors = cmap(norm(sim_params.t0[p]))
            print("start time embryos", sim_params.t0[p])
            axs[k].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
            cmap = cmap)

            if iso_idx != 0:
                axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
                                color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

            axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                        color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
            
    print(k, planet_counter(simulations, parameters, sim_parameters))
    # for i in range(1):
    #     for p in [0]:
    #         idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
    #                         sim.dR_dt[p], sim.dM_dt[p], params, True)
    #         iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
    #         stop_mig_idx = idx['stop_mig_idx'].values[0]
    #         dt = (sim.time[:iso_idx]).to(u.Myr)
    #         sc = axs[k].scatter(sim.position[p,:iso_idx].to(u.au), sim.mass[p,:iso_idx].to(u.M_earth), c = dt, 
    #                             norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), 
    #         cmap = mpl.cm.inferno.reversed())
    #         axs[k].loglog(sim.position.to(u.au)[p,iso_idx:stop_mig_idx], sim.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, 
    #                     alpha = alpha_transp)
    #         axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
    #         axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.earthMass), 
    #                     color = cmap(norm(sim.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
    axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
    axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
    axs[k].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)


cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(sc, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker

for j in range(3):
    # #SE
    axs[j].fill_betweenx([1, 20], 0.001, 1, color='brown', alpha=0.1)
    #terrestrial embryo
    axs[j].fill_betweenx([0.01, 1], 0.1, 10, color='black', alpha=0.1)

    # gas giants
    axs[j].fill_betweenx([1e2, 6e3], 0.001, 0.1, color='aquamarine', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 0.1, 2, color='gold', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 2, 10, color='purple', alpha=0.1)


    plot.HD219134(axs[j], color = 'red')
    plot.solar_system(axs[j], color = 'green')
    axs[j].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

    axs[j].set_xlabel('r [AU]', fontsize = 20, labelpad=20)
    axs[j].set_title(title[j], fontsize = 20)
    axs[j].set_xlim(5e-3, 1e2)
    axs[j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[j].set_ylim(5e-8, 1e3)

    axs[j].yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs[j].yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 20, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_gas_models_longer_t_3Myr", bbox_inches='tight')